# 재생성 작업 만들기

## DAG 실패시 재생성하는 Loop 만들기

작업 환경 : wavvedp-prd restore jupyterNotebook
- notes : `src/prd_rho/restore/restore_wavve_log_notes/notes_process_all.ipynb`
- zzim : `src/prd_rho/restore/restore_wavve_log_zzim/zzim_process_all.ipynb`

In [ ]:


from datetime import datetime, timedelta

date1 = "2022-12-01 06:00:00"
date2 = "2022-12-01 08:00:00"

start_from = datetime.strptime(date1, '%Y-%m-%d %H:%M:%S')
end_to = datetime.strptime(date2, '%Y-%m-%d %H:%M:%S')
print(start_from, " ~ " ,end_to)


while start_from < end_to :
    
    yyyy_mm_dd, yyyymmdd = (start_from.strftime('%Y-%m-%d'), start_from.strftime('%Y%m%d'))
    yyyy,mm,dd = (yyyymmdd[:4], yyyymmdd[4:6], yyyymmdd[6:]) 
    hh = start_from.strftime('%H')
    #print(yyyy_mm_dd, hh)

    start_time = '{start_time} {hh}:00:00'.format(start_time = yyyy_mm_dd, hh = hh) #2022-08-05 00:00:00
    delta_hour = timedelta(hours=1)
    end_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + delta_hour #2022-08-05 01:00:00

    print("\n# ****************************************************************************************************************** ")
    
    print("* job date from \n* ", start_time, " ~ ", end_time)
    
    print("\n# ################################################################################################################## wavvedp-prd : HAYSTACK #####")
    # enviornment setting
    warehouse_project, project, dataset, datatable = ('warehouse_prd', 'wavvedp-prd', 'haystack', 'notes')

    # haystack backup
    query_prd_haystack_backup = f"""
    # ----------------------------------------------------------------------
    EXPORT DATA OPTIONS (
        uri = 'gs://{warehouse_project}/{dataset}/{datatable}/yyyy={yyyy}/mm={mm}/dd={dd}/hh={hh}/*.parquet',
        format = 'PARQUET',
        overwrite = true,
        compression = 'GZIP')
    AS (
        SELECT *
        FROM `{project}.{dataset}.{datatable}`
        WHERE created_at >= '{start_time}+09:00' and created_at < '{end_time}+09:00'
    );
    """
                 

    print(query_prd_haystack_backup)
    
    print("\n# ################################################################################################################## wavvedp-prd : NOTES #####")
    project, dataset, table_name = ('wavvedp-prd', 'wavve_log', 'notes')

    # prd notes
    query_prd_notes_create = f"""
    CREATE TABLE IF NOT EXISTS `wavvedp-prd.wavve_log.notes_{yyyymmdd}`
    (
      log_version STRING OPTIONS(description="로그분류"),
      ap_timestamp TIMESTAMP OPTIONS(description="서버로그시간 KST"),
      log_timestamp STRING OPTIONS(description="클라이언트로그시간 string KST"),
      uno STRING OPTIONS(description="사용자고유번호"),
      profile_id STRING OPTIONS(description="사용자프로필고유번호"),
      zone STRING OPTIONS(description="servicezonetype"),
      device_type STRING OPTIONS(description="클라이언트단말구분"),
      agent STRING OPTIONS(description="클라이언트agent정보"),
      app_version STRING OPTIONS(description="appversion"),
      service STRING OPTIONS(description="Log가발생한서비스정의"),
      log_type STRING OPTIONS(description="로그타입-md:미디어전송,ad:광고전송"),
      item_type STRING OPTIONS(description="화질구분"),
      channel_type STRING OPTIONS(description="재생콘텐츠구분-L:Live,V:VOD,M:Movie,C:Clip,E:Extra"),
      program_id STRING OPTIONS(description="프로그램고유ID"),
      content_id STRING OPTIONS(description="콘텐츠고유ID"),
      corner_id STRING OPTIONS(description="코너고유ID"),
      ap_date STRING OPTIONS(description="서버전송시간"),
      media_time STRING OPTIONS(description="미디어진행시간"),
      guid STRING OPTIONS(description="클라이언트단말고유ID"),
      play_id STRING OPTIONS(description="재생구분고유ID"),
      concurrency_group STRING OPTIONS(description="동시시청그룹"),
      issue_timestamp STRING OPTIONS(description="재생URL발급시각 string KST"),
      is_abr STRING OPTIONS(description="ABR여부"),
      br STRING OPTIONS(description="현재재생bitrate"),
      is_charged STRING OPTIONS(description="유료사용자여부"),
      price_type STRING OPTIONS(description="콘텐츠가격구분"),
      ip_address STRING OPTIONS(description="클라이언트IP주소"),
      extra STRING OPTIONS(description="기타정보")
    )
    PARTITION BY TIMESTAMP_TRUNC(ap_timestamp, HOUR)
    OPTIONS(
      require_partition_filter=true,
      description="Notes 데이터 저장"
    );
    """
    print(query_prd_notes_create)
    
    query_prd_notes_insert = f"""
    # ----------------------------------------------------------------------

    DELETE `wavvedp-prd.wavve_log.notes_{yyyymmdd}`
    WHERE ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}';

    # ----------------------------------------------------------------------

    INSERT INTO `wavvedp-prd.wavve_log.notes_{yyyymmdd}`
    SELECT 
      -- common log
      json_extract_scalar(data, '$.log_version') `log_version`,
      TIMESTAMP_ADD(TIMESTAMP(created_at), INTERVAL 9 HOUR) `ap_timestamp`, -- 로그 서버 시간 KST로 변환
      json_extract_scalar(data, '$.log_timestamp') `log_timestamp`,
      json_extract_scalar(data, '$.uno') `uno`,
      json_extract_scalar(data, '$.profileid') `profile_id`,
      json_extract_scalar(data, '$.zone') `zone`,
      json_extract_scalar(data, '$.device') `device_type`,
      json_extract_scalar(data, '$.agent') `agent`,
      json_extract_scalar(data, '$.app_version') `app_version`,
      json_extract_scalar(data, '$.service') `service`,

      -- note log
      json_extract_scalar(data, '$.data.common.logType') `log_type`,  -- ad : 광고, md : 미디어 콘텐츠
      json_extract_scalar(data, '$.data.common.itemType') `item_type`, -- 화질 구분
      json_extract_scalar(data, '$.data.common.channelType') `channel_type`, -- L : Live, V : VOD, M : Movie, C : Clip, E : Extra
      json_extract_scalar(data, '$.data.common.programId') `program_id`,
      json_extract_scalar(data, '$.data.common.contentId') `content_id`,
      json_extract_scalar(data, '$.data.common.cornerId') `corner_id`,
      json_extract_scalar(data, '$.data.common.apDate') `ap_date`, -- apDate null인 값들이 있음
      json_extract_scalar(data, '$.data.common.mediaTime') `media_time`, -- 미디어 진행 시간
      json_extract_scalar(data, '$.data.common.guid') `guid`, -- 클라이언트 단말 고유 ID
      json_extract_scalar(data, '$.data.common.playId') `play_id`, -- 재생 구분 고유 ID
      json_extract_scalar(data, '$.data.common.concurrencyGroup') `concurrency_group`, -- 동시시청 그룹, 0: 동시시청제외, 1 ∼ n: 동시시청 그룹
      json_extract_scalar(data, '$.data.common.issue') `issue_timestamp`,
      json_extract_scalar(data, '$.data.common.isABR') `is_abr`, -- Adaptive Bitrate 사용 여부: Y , N 
      json_extract_scalar(data, '$.data.common.BR') `br`, -- 현재 재생 bitrate 
      json_extract_scalar(data, '$.data.common.isCharged') `is_charged`,  -- 유료사용자 여부: Y , N 
      json_extract_scalar(data, '$.data.common.priceType') `price_type`,  -- P: 유료콘텐츠, F: 무료콘텐츠, N: 일부화질무료 
      json_extract_scalar(data, '$.data.common.ipAddress') `ip_address`,  -- 클라이언트IP 주소 

      -- 추가 데이터
      json_extract(data, '$.data.extra') `extra` -- apiV, api version
    FROM `wavvedp-prd.haystack.notes`
    WHERE
      created_at >= '{start_time}+09:00' and created_at < '{end_time}+09:00'
      and json_extract_scalar(data, '$.uno') != 'test'
      and json_extract_scalar(data, '$.log_type') = 'notes'
    """
    print(query_prd_notes_insert)
    
    print("\n# ################################################################################################################## wavvedp-dw : NOTES #####")
    project, dataset, table_name = ('wavvedp-dw', 'wavve_log', 'notes')

    # dw notes
    query_dw_notes_create = f"""
    CREATE TABLE IF NOT EXISTS `wavvedp-dw.wavve_log.notes_{yyyymmdd}`
    (
      log_version STRING OPTIONS(description="로그분류"),
      ap_timestamp TIMESTAMP OPTIONS(description="서버로그시간 KST"),
      log_timestamp STRING OPTIONS(description="클라이언트로그시간 string KST"),
      uno STRING OPTIONS(description="사용자고유번호"),
      profile_id STRING OPTIONS(description="사용자프로필고유번호"),
      zone STRING OPTIONS(description="servicezonetype"),
      device_type STRING OPTIONS(description="클라이언트단말구분"),
      agent STRING OPTIONS(description="클라이언트agent정보"),
      app_version STRING OPTIONS(description="appversion"),
      service STRING OPTIONS(description="Log가발생한서비스정의"),
      log_type STRING OPTIONS(description="로그타입-md:미디어전송,ad:광고전송"),
      item_type STRING OPTIONS(description="화질구분"),
      channel_type STRING OPTIONS(description="재생콘텐츠구분-L:Live,V:VOD,M:Movie,C:Clip,E:Extra"),
      program_id STRING OPTIONS(description="프로그램고유ID"),
      content_id STRING OPTIONS(description="콘텐츠고유ID"),
      corner_id STRING OPTIONS(description="코너고유ID"),
      ap_date STRING OPTIONS(description="서버전송시간"),
      media_time STRING OPTIONS(description="미디어진행시간"),
      guid STRING OPTIONS(description="클라이언트단말고유ID"),
      play_id STRING OPTIONS(description="재생구분고유ID"),
      concurrency_group STRING OPTIONS(description="동시시청그룹"),
      issue_timestamp STRING OPTIONS(description="재생URL발급시각 string KST"),
      is_abr STRING OPTIONS(description="ABR여부"),
      br STRING OPTIONS(description="현재재생bitrate"),
      is_charged STRING OPTIONS(description="유료사용자여부"),
      price_type STRING OPTIONS(description="콘텐츠가격구분"),
      ip_address STRING OPTIONS(description="클라이언트IP주소"),
      extra STRING OPTIONS(description="기타정보")
    )
    PARTITION BY TIMESTAMP_TRUNC(ap_timestamp, HOUR)
    OPTIONS(
      require_partition_filter=true,
      description="Notes 데이터 저장"
    );
    """
    print(query_dw_notes_create)

    query_dw_notes_insert = f"""
    # ----------------------------------------------------------------------

    DELETE `wavvedp-dw.wavve_log.notes_{yyyymmdd}`
    WHERE ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}';

    # ----------------------------------------------------------------------

    INSERT INTO `wavvedp-dw.wavve_log.notes_{yyyymmdd}`
    SELECT 
        log_version,
        ap_timestamp,
        log_timestamp,
        TO_HEX(SHA512(CONCAT('WAVVE', uno))) uno,
        profile_id,
        zone,
        device_type,
        agent,
        app_version,
        service,
        log_type,
        item_type,
        channel_type,
        program_id,
        content_id,
        corner_id,
        ap_date,
        media_time,
        guid,
        play_id,
        concurrency_group,
        issue_timestamp,
        is_abr,
        br,
        is_charged,
        price_type,
        '' as ip_address,
        extra
    FROM `wavvedp-prd.wavve_log.notes_{yyyymmdd}`
    WHERE ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}'
    """
    print(query_dw_notes_insert)
    
    print("\n# ############################################################################################################ wavvedp-prd : NOTES_HOURLY #####")
    project, dataset, table_name = ('wavvedp-prd', 'wavve_log', 'notes_hourly')

    # prd notes_hourly
    query_prd_notes_hourly_create = f"""
    CREATE TABLE IF NOT EXISTS `wavvedp-prd.wavve_log.notes_hourly_{yyyymmdd}`
    (
      program_id STRING OPTIONS(description="program id"),
      content_id STRING OPTIONS(description="content id"),
      uno STRING OPTIONS(description="user number"),
      profile_id STRING OPTIONS(description="user profile id"),
      device_type STRING OPTIONS(description="클라이언트 단말 구분"),
      is_charged STRING OPTIONS(description="유료사용자 여부"),
      item_type STRING OPTIONS(description="화질 구분"),
      zone STRING OPTIONS(description="wavveon 구분"),
      price_type STRING OPTIONS(description="콘텐츠 가격 구분"),
      pass STRING OPTIONS(description="이용권 - pass"),
      item STRING OPTIONS(description="이용권 - item"),
      package STRING OPTIONS(description="이용권 - package"),
      product STRING OPTIONS(description="이용권 product"),
      etc STRING OPTIONS(description="이용권 - 나머지"),
      channel_type STRING OPTIONS(description="재생 콘텐츠 구분"),
      view_time INT64 OPTIONS(description="시청시간(초)"),
      ap_datetime DATETIME OPTIONS(description="파티션 컬럼"),
      yyyy_mm_dd DATE OPTIONS(description="로그 날짜 - DATE"),
      hh STRING OPTIONS(description="로그 시간"),
      yyyymmdd STRING OPTIONS(description="로그 날짜 - 문자열")
    )
    PARTITION BY DATETIME_TRUNC(ap_datetime, HOUR)
    CLUSTER BY hh, program_id, content_id
    OPTIONS(
      require_partition_filter=true,
      description="Hourly 집계 Notes 데이터 저장"
    );
    """
    
    print(query_prd_notes_hourly_create)

    query_prd_notes_hourly_insert = f"""
    # ----------------------------------------------------------------------

    DELETE `wavvedp-prd.wavve_log.notes_hourly_{yyyymmdd}`
    WHERE ap_datetime >= '{start_time}' and ap_datetime < '{end_time}';

    # ----------------------------------------------------------------------

    INSERT INTO `wavvedp-prd.wavve_log.notes_hourly_{yyyymmdd}`
    SELECT
        program_id,
        content_id,
        uno,
        profile_id,
        device_type,
        is_charged,
        item_type,
        zone,
        price_type,
        if(instr(B.product, 'pass') > 1, replace(B.product, '=pass', ''), '') pass, -- pass
        if(instr(B.product, 'item') > 1, replace(B.product, '=item', ''), '') item, -- item
        if(instr(B.product, 'package') > 1, replace(B.product, '=package', ''), '') package, -- package
        if(instr(B.product, 'product') > 1, replace(B.product, '=product', ''), '') product, -- product
        IF(instr(B.product, 'pass') = -1 AND instr(B.product, 'item') = -1 AND instr(B.product, 'package') = -1 AND instr(B.product, 'product') = -1, B.product, '') etc,
        channel_type,
        count(*) * 10 view_time,
        ap_datetime,
        yyyy_mm_dd,
        hh,
        yyyymmdd
    FROM (
        SELECT
        program_id,
        content_id,
        uno,
        profile_id,
        device_type,
        UPPER(is_charged) is_charged,
        item_type,
        zone,
        UPPER(price_type) price_type,
        ARRAY_TO_STRING(ARRAY(SELECT distinct p FROM unnest(split(json_extract_scalar(extra, '$.product'), ',')) p WHERE p != '' ), ',') product,
        channel_type,
        PARSE_DATETIME("%Y-%m-%d %H:00:00", format_timestamp("%Y-%m-%d %H:00:00", ap_timestamp)) ap_datetime,
        DATE(format_timestamp("%Y-%m-%d %H:00:00", ap_timestamp)) yyyy_mm_dd,
        format_timestamp("%H", ap_timestamp) `hh`,
        format_timestamp("%Y%m%d", ap_timestamp) yyyymmdd
        FROM `wavvedp-prd.wavve_log.notes_{yyyymmdd}`
        WHERE
        ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}'
        ) B
    GROUP BY
        program_id,  content_id,  uno, profile_id, device_type, is_charged, item_type, zone, price_type,
        pass, item, package, product, etc, channel_type,
        ap_datetime,
        yyyy_mm_dd,
        hh,
        yyyymmdd
    """
    
    print(query_prd_notes_hourly_insert)
    
    print("\n# ############################################################################################################ wavvedp-dw : NOTES_HOURLY #####")
    
    project, dataset, table_name = ('wavvedp-dw', 'wavve_log', 'notes_hourly')

    # dw notes_hourly
    query_dw_notes_hourly_create = f"""
    CREATE TABLE IF NOT EXISTS `wavvedp-dw.wavve_log.notes_hourly_{yyyymmdd}`
    (
      program_id STRING OPTIONS(description="program id"),
      content_id STRING OPTIONS(description="content id"),
      uno STRING OPTIONS(description="user number"),
      profile_id STRING OPTIONS(description="user profile id"),
      device_type STRING OPTIONS(description="클라이언트 단말 구분"),
      is_charged STRING OPTIONS(description="유료사용자 여부"),
      item_type STRING OPTIONS(description="화질 구분"),
      zone STRING OPTIONS(description="wavveon 구분"),
      price_type STRING OPTIONS(description="콘텐츠 가격 구분"),
      pass STRING OPTIONS(description="이용권 - pass"),
      item STRING OPTIONS(description="이용권 - item"),
      package STRING OPTIONS(description="이용권 - package"),
      product STRING OPTIONS(description="이용권 product"),
      etc STRING OPTIONS(description="이용권 - 나머지"),
      channel_type STRING OPTIONS(description="재생 콘텐츠 구분"),
      view_time INT64 OPTIONS(description="시청시간(초)"),
      ap_datetime DATETIME OPTIONS(description="파티션 컬럼"),
      yyyy_mm_dd DATE OPTIONS(description="로그 날짜 - DATE"),
      hh STRING OPTIONS(description="로그 시간"),
      yyyymmdd STRING OPTIONS(description="로그 날짜 - 문자열")
    )
    PARTITION BY DATETIME_TRUNC(ap_datetime, HOUR)
    CLUSTER BY hh, program_id, content_id
    OPTIONS(
      require_partition_filter=true,
      description="Notes - Hourly 집계 데이터 저장"
    );
    """
    
    print(query_dw_notes_hourly_create)
    
    query_dw_notes_hourly_insert = f"""
    # ----------------------------------------------------------------------

    DELETE `wavvedp-dw.wavve_log.notes_hourly_{yyyymmdd}`
    WHERE ap_datetime >= '{start_time}' and ap_datetime < '{end_time}';

    # ----------------------------------------------------------------------

    INSERT INTO `wavvedp-dw.wavve_log.notes_hourly_{yyyymmdd}`
    SELECT
        program_id,
        content_id,
        TO_HEX(SHA512(CONCAT('WAVVE', uno))) uno,
        profile_id,
        device_type,
        is_charged,
        item_type,
        zone,
        price_type,
        pass,
        item,
        package,
        product,
        etc,
        channel_type,
        view_time,
        ap_datetime,
        yyyy_mm_dd,
        hh,
        yyyymmdd
    FROM `wavvedp-prd.wavve_log.notes_hourly_{yyyymmdd}`
    WHERE ap_datetime >= '{start_time}' and ap_datetime < '{end_time}'

    """
    
    print(query_dw_notes_hourly_insert)
    
    
    start_from += delta_hour # 2022-08-05 -> 2022-08-04

In [ ]:
from datetime import datetime, timedelta

date1 = "2022-12-01 06:00:00"
date2 = "2022-12-01 08:00:00"

start_from = datetime.strptime(date1, '%Y-%m-%d %H:%M:%S')
end_to = datetime.strptime(date2, '%Y-%m-%d %H:%M:%S')
print(start_from, " ~ " ,end_to)


while start_from < end_to :
    
    yyyy_mm_dd, yyyymmdd = (start_from.strftime('%Y-%m-%d'), start_from.strftime('%Y%m%d'))
    yyyy,mm,dd = (yyyymmdd[:4], yyyymmdd[4:6], yyyymmdd[6:]) 
    hh = start_from.strftime('%H')
    #print(yyyy_mm_dd, hh)

    start_time = '{start_time} {hh}:00:00'.format(start_time = yyyy_mm_dd, hh = hh) #2022-08-05 00:00:00
    delta_hour = timedelta(hours=1)
    end_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + delta_hour #2022-08-05 01:00:00

    print("\n# ****************************************************************************************************************** ")
    
    print("* job date from \n* ", start_time, " ~ ", end_time)
    
    print("\n# ################################################################################################################## wavvedp-prd : HAYSTACK #####")
    # enviornment setting
    warehouse_project, project, dataset, datatable = ('warehouse_prd', 'wavvedp-prd', 'haystack', 'favorites')

    # haystack backup
    query_prd_haystack_backup = f"""
    # ----------------------------------------------------------------------
    EXPORT DATA OPTIONS (
        uri = 'gs://{warehouse_project}/{dataset}/{datatable}/yyyy={yyyy}/mm={mm}/dd={dd}/hh={hh}/*.parquet',
        format = 'PARQUET',
        overwrite = true,
        compression = 'GZIP')
    AS (
        SELECT *
        FROM `{project}.{dataset}.{datatable}`
        WHERE created_at >= '{start_time}+09:00' and created_at < '{end_time}+09:00'
    );
    """
                 

    print(query_prd_haystack_backup)
    
    print("\n# ################################################################################################################## wavvedp-prd : zzim #####")
    project, dataset, table_name = ('wavvedp-prd', 'wavve_log', 'zzim')

    # prd zzim
    query_prd_zzim_create = f"""
    CREATE TABLE IF NOT EXISTS `wavvedp-prd.wavve_log.zzim_{yyyymmdd}`
    (
      log_type STRING OPTIONS(description="zzim"),
      log_timestamp STRING OPTIONS(description="클라이언트로그시간 string KST"),
      ap_timestamp TIMESTAMP OPTIONS(description="서버로그시간 KST"),
      uno STRING OPTIONS(description="사용자고유번호"),
      profile_id STRING OPTIONS(description="사용자프로필고유번호"),
      zone STRING OPTIONS(description="service zone type"),
      device_type STRING OPTIONS(description="클라이언트단말구분"),
      agent STRING OPTIONS(description="클라이언트agent정보"),
      app_version STRING OPTIONS(description="app version"),
      action_type STRING OPTIONS(description="찜 등록/해제 액션 타입(register, delete, delete_all)"),
      content_type STRING OPTIONS(description="찜 등록/해제한 콘텐츠 타입 - ‘live, ‘movie, 'program’, 'theme’"),
      content_id STRING OPTIONS(description="찜 등록/해제한 콘텐츠 id - ‘live-channel_id, ‘movie-movie_id, 'program-program_id’, 'theme-ui_code’")
    )
    PARTITION BY TIMESTAMP_TRUNC(ap_timestamp, HOUR)
    OPTIONS(
      require_partition_filter=true,
      description="ZZIM(favorites) 데이터 저장"
    );
    """
    print(query_prd_zzim_create)
    
    query_prd_zzim_insert = f"""
    # ----------------------------------------------------------------------

    DELETE `wavvedp-prd.wavve_log.zzim_{yyyymmdd}`
    WHERE ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}';

    # ----------------------------------------------------------------------

    INSERT INTO `wavvedp-prd.wavve_log.zzim_{yyyymmdd}`
    WITH unnest_action_value as (
      SELECT
        JSON_QUERY_ARRAY(data, "$.data.action_value") `action_value_array` --array
        , created_at
        , data
      FROM `wavvedp-prd.haystack.favorites`
      WHERE
      created_at >= '{start_time}+09:00' and created_at < '{end_time}+09:00'
      and json_extract_scalar(data, '$.uno') != 'test'
      and json_extract_scalar(data, '$.log_type') != '1.1.1'
      and json_extract_scalar(data, '$.log_type') = 'zzim'
    )
    select
         -- common log
         json_extract_scalar(data, '$.log_type') `log_type`  -- zzim
         , json_extract_scalar(data, '$.log_timestamp') `log_timestamp`
         , TIMESTAMP_ADD(TIMESTAMP(created_at), INTERVAL 9 HOUR) `ap_timestamp` -- 로그 서버 시간 KST로 변환
         , json_extract_scalar(data, '$.uno') `uno`
         , json_extract_scalar(data, '$.profileid') `profile_id`
         , json_extract_scalar(data, '$.zone') `zone`
         , json_extract_scalar(data, '$.device') `device_type`
         , json_extract_scalar(data, '$.agent') `agent`
         , json_extract_scalar(data, '$.app_version') `app_version`

         , json_extract_scalar(data, '$.data.action_type') `action_type`
         --action_value
         , json_extract_scalar(action_value, '$.content_type') `content_type`
         , json_extract_scalar(action_value, '$.content_id') `content_id`

    from unnest_action_value, unnest(unnest_action_value.action_value_array) as action_value
    """
    print(query_prd_zzim_insert)
    
    print("\n# ################################################################################################################## wavvedp-dw : zzim #####")
    project, dataset, table_name = ('wavvedp-dw', 'wavve_log', 'zzim')

    # dw zzim
    query_dw_zzim_create = f"""
    CREATE TABLE IF NOT EXISTS `wavvedp-dw.wavve_log.zzim_{yyyymmdd}`
    (
      log_type STRING OPTIONS(description="zzim"),
      log_timestamp STRING OPTIONS(description="클라이언트로그시간 string KST"),
      ap_timestamp TIMESTAMP OPTIONS(description="서버로그시간 KST"),
      uno STRING OPTIONS(description="사용자고유번호"),
      profile_id STRING OPTIONS(description="사용자프로필고유번호"),
      zone STRING OPTIONS(description="service zone type"),
      device_type STRING OPTIONS(description="클라이언트단말구분"),
      agent STRING OPTIONS(description="클라이언트agent정보"),
      app_version STRING OPTIONS(description="app version"),
      action_type STRING OPTIONS(description="찜 등록/해제 액션 타입(register, delete, delete_all)"),
      content_type STRING OPTIONS(description="찜 등록/해제한 콘텐츠 타입 - ‘live, ‘movie, 'program’, 'theme’"),
      content_id STRING OPTIONS(description="찜 등록/해제한 콘텐츠 id - ‘live-channel_id, ‘movie-movie_id, 'program-program_id’, 'theme-ui_code’")
    )
    PARTITION BY TIMESTAMP_TRUNC(ap_timestamp, HOUR)
    OPTIONS(
      require_partition_filter=true,
      description="ZZIM(favorites) 데이터 저장"
    );
    """
    print(query_dw_zzim_create)

    query_dw_zzim_insert = f"""
    # ----------------------------------------------------------------------

    DELETE `wavvedp-dw.wavve_log.zzim_{yyyymmdd}`
    WHERE ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}';

    # ----------------------------------------------------------------------

    INSERT INTO `wavvedp-dw.wavve_log.zzim_{yyyymmdd}`
    SELECT
        log_type,
        log_timestamp,
        ap_timestamp,
        TO_HEX(SHA512(CONCAT('WAVVE', uno))) uno,
        profile_id,
        zone,
        device_type,
        agent,
        app_version,
        action_type,
        content_type,
        content_id
    FROM `wavvedp-prd.wavve_log.zzim_{yyyymmdd}`
    WHERE ap_timestamp >= '{start_time}' and ap_timestamp < '{end_time}'
    """
    print(query_dw_zzim_insert)
    
    
    
    start_from += delta_hour # 2022-08-05 -> 2022-08-04

# Title

## 설명